# Khoa CNTT K19

In [22]:
import os
import pdfplumber

# Thư mục chứa các file PDF
folder_dt = "./K19CNTT_DT"  # 🛑 Hãy thay bằng đường dẫn thật, ví dụ: "C:/Users/you/Downloads/pdfs"
folder_clc = "./K19CNTT_CLC"
mssv_dt = []
mssv_clc = []

def extract_mssv_from_folder(folder_path):
    mssv_list = []
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(".pdf"):
            file_path = os.path.join(folder_path, filename)
            print(f"Đang đọc file: {file_path}")
            try:
                with pdfplumber.open(file_path) as pdf:
                    for page in pdf.pages:
                        tables = page.extract_tables()
                        for table in tables:
                            for row in table:
                                # Bỏ qua nếu row không hợp lệ hoặc không đủ cột
                                if row and len(row) > 1 and row[1] and isinstance(row[1], str) and row[1].strip().isdigit():
                                    mssv_list.append(row[1].strip())
            except Exception as e:
                print(f"Lỗi khi đọc {filename}: {e}")
    return mssv_list

mssv_dt = extract_mssv_from_folder(folder_dt)
mssv_clc = extract_mssv_from_folder(folder_clc)

len(mssv_dt), len(mssv_clc)

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


Đang đọc file: ./K19CNTT_DT\CNTT_K19_dai tra.pdf


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


Đang đọc file: ./K19CNTT_DT\He thong thong tin_K19_DAI TRA(1).pdf


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


...

Đang đọc file: ./K19CNTT_CLC\KHOA HOC MAY TINH_K19_CLC.pdf


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


Đang đọc file: ./K19CNTT_CLC\KTPM_K19_CLC.pdf


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


(474, 368)

## Cell này chạy khoảng 25p

In [23]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.options import Options
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import csv

# --- CẤU HÌNH ẨN TRÌNH DUYỆT ---
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")

driver = webdriver.Edge(options=options)
driver.get("https://pdt.iuh.edu.vn/tra-cuu-hoc-bong/")

wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.ID, "maSV")))

# Chọn học kỳ và năm học
Select(driver.find_element(By.ID, "hocky")).select_by_value("2")
driver.find_element(By.ID, "namhoc").send_keys("2023-2024")

# --- HÀM PARSE HTML ---
def parse_result(driver):
    wait.until(lambda d: d.find_elements(By.CSS_SELECTOR, "#search_results .mydiv")
               or "Không tìm thấy" in d.find_element(By.ID, "search_results").text)

    container = driver.find_element(By.ID, "search_results")
    inner_html = container.get_attribute("innerHTML")
    soup = BeautifulSoup(inner_html, "html.parser")

    if "Không tìm thấy" in soup.text:
        return None

    data = {}
    for div in soup.select(".mydiv"):
        label_span = div.select_one("span.sp_1")
        if not label_span:
            continue
        label = label_span.get_text(strip=True).rstrip(":")
        text_full = div.get_text(" ", strip=True)
        value = text_full.replace(label_span.get_text(), "", 1).strip().lstrip(":").strip()
        data[label] = value
    return data

# --- VÒNG LẶP QUA DANH SÁCH SINH VIÊN ---
def temp(output_fn, mssv_list):
    print(f"🔎 Đang xử lý...")
    fieldnames = [
        "MSSV", "Mã số sinh viên", "Họ tên", "Lớp",
        "Điểm", "Số TC", "Mức học bổng",
        "Xếp loại học tập", "Xếp loại rèn luyện",
        "Số tiền", "Ghi chú"
    ]
    with open(output_fn, "w", encoding="utf-8", newline="") as fout:
        writer = csv.DictWriter(fout, fieldnames=fieldnames)
        writer.writeheader()
    
        for mssv in mssv_list:
            inp = driver.find_element(By.ID, "maSV")
            inp.clear()
            inp.send_keys(mssv)
            driver.find_element(By.ID, "timkiem_btn").click()
        
            # CHỜ cập nhật kết quả
            time.sleep(1.5)
            result = parse_result(driver)
            if result:
                row = {k: result.get(k, "") for k in fieldnames}
                row["MSSV"] = mssv
                writer.writerow(row)

temp("./HB_K19CNTT/HB_K19CNTT_DT.csv", mssv_dt)
temp("./HB_K19CNTT/HB_K19CNTT_CLC.csv", mssv_clc)

print("✅ Hoàn tất.")
driver.quit()

🔎 Đang xử lý...
🔎 Đang xử lý...
✅ Hoàn tất.


In [41]:
import pandas as pd

# Đọc dữ liệu
df_dt = pd.read_csv("./HB_K19CNTT/HB_K19CNTT_DT.csv")
df_clc = pd.read_csv("./HB_K19CNTT/HB_K19CNTT_CLC.csv")

# Thêm cột chương trình
df_dt["Chương trình"] = "Đại trà"
df_clc["Chương trình"] = "CLC"

# Gộp dữ liệu
df = pd.concat([df_dt, df_clc], ignore_index=True)

# Làm sạch số tiền
df["Số tiền"] = (
    df["Số tiền"]
    .str.replace(".", "", regex=False)
    .str.replace(",", "", regex=False)
    .str.extract(r"(\d+)")
    .astype(float)
)

# Tổng số SV thực tế
total_students = 842
num_dt = len(df_dt)
num_clc = len(df_clc)
num_total = len(df)
percent_total = num_total / total_students * 100

print(f"📌 Số SV có học bổng:")
print(f"  - Đại trà: {num_dt}")
print(f"  - CLC: {num_clc}")
print(f"  - Tổng cộng: {num_total} ({percent_total:.2f}%)\n")

# Mức học bổng cần thống kê
muc_hoc_bong_levels = ["30%", "50%", "70%", "100%"]

print("📊 Thống kê chi tiết từng mức học bổng cho Đại trà và CLC:\n")

# Duyệt theo từng chương trình và mức học bổng
for program in ["Đại trà", "CLC"]:
    print(f"🔷 Chương trình: {program}")
    df_prog = df[df["Chương trình"] == program]
    
    for muc in muc_hoc_bong_levels:
        subset = df_prog[df_prog["Mức học bổng"] == muc]
        so_sv = len(subset)
        ty_le = so_sv / total_students * 100
        if so_sv > 0:
            diem_min = subset["Điểm"].min()
            diem_max = subset["Điểm"].max()
            tien = subset["Số tiền"].iloc[0]
            print(f"  - Mức {muc}: {so_sv} SV ({ty_le:.2f}%)")
            print(f"      + Khoảng điểm: {diem_min} - {diem_max}")
            print(f"      + Số tiền: {tien:,.0f} VNĐ")
        else:
            print(f"  - Mức {muc}: Không có SV")
    print()


📌 Số SV có học bổng:
  - Đại trà: 119
  - CLC: 30
  - Tổng cộng: 149 (17.70%)

📊 Thống kê chi tiết từng mức học bổng cho Đại trà và CLC:

🔷 Chương trình: Đại trà
  - Mức 30%: 27 SV (3.21%)
      + Khoảng điểm: 3.45 - 3.51
      + Số tiền: 4,656,000 VNĐ
  - Mức 50%: 30 SV (3.56%)
      + Khoảng điểm: 3.52 - 3.56
      + Số tiền: 7,760,000 VNĐ
  - Mức 70%: 27 SV (3.21%)
      + Khoảng điểm: 3.5 - 3.67
      + Số tiền: 10,864,000 VNĐ
  - Mức 100%: 35 SV (4.16%)
      + Khoảng điểm: 3.69 - 3.93
      + Số tiền: 15,520,000 VNĐ

🔷 Chương trình: CLC
  - Mức 30%: 11 SV (1.31%)
      + Khoảng điểm: 3.45 - 3.5
      + Số tiền: 5,782,500 VNĐ
  - Mức 50%: 5 SV (0.59%)
      + Khoảng điểm: 3.53 - 3.56
      + Số tiền: 9,637,500 VNĐ
  - Mức 70%: 11 SV (1.31%)
      + Khoảng điểm: 3.57 - 3.68
      + Số tiền: 13,492,500 VNĐ
  - Mức 100%: 3 SV (0.36%)
      + Khoảng điểm: 3.69 - 3.78
      + Số tiền: 19,275,000 VNĐ

